In [1]:
import numpy as np
import math
import csv
import pandas as pd
import os
import time
from tqdm import tqdm
from collections import Counter
import seaborn as sns
import pickle

In [2]:
df_result = pd.read_pickle('/Users/s0c02nj/Desktop/Item_search_Std_Ref/')

In [3]:
df_result.head()

,Ingredients_cln,Ingredients_org,UPC_match,Signing_Desc_Match,Qarth_Desc_Match,list_score_qarth
0,habanero extract,HABANERO EXTRACT,85761000560,HABANERO 4OZ,habanero,10.1642
1,habanero extract,HABANERO EXTRACT,1996235125,HABANERO 4OZ,habanero,10.1642
2,habanero extract,HABANERO EXTRACT,40008464060,CINTA #9 50YARD CON DISEO,habanero,10.1642
3,habanero extract,HABANERO EXTRACT,40555164658,NAME HABANERO,habanero,10.1642
4,habanero extract,HABANERO EXTRACT,71106953272,HABANERO 2OZ,habanero,10.1642


In [4]:
df_result['Ingredients_cln'].unique().shape

(28536,)

In [ ]:
def get_cosine(str1, str2):
    
    #Counter forms a dictionary----> Frequency of words as values and key is the word
    vec1 = Counter(str1.split())
    vec2 = Counter(str2.split())
    
    #Intersection contains the set of common words
    intersection = set(vec1.keys()) & set(vec2.keys())
    
    #W1W2W2=[1,2,3] and W1W2W3=[2,4,6] ,it finds the a.b value
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    #|a| and |b| are computed through the process
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    #cosine value is computed
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [ ]:
#Getting Jaccard Distance
def DistJaccard(str1, str2):
    str1 = set(str1.split())
    str2 = set(str2.split())
    #a_intersection_b ---unique
    numerator = len(str1 & str2)
    #a_union_b---unique
    denominator = min(len(str1),len(str2))
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


In [ ]:
jacc_sim = []
cosine_sim = []

for i in tqdm(range(0,len(df_result))):
    
    inred = df_result['Ingredients_cln'].iloc[i]
    match_desc = df_result['Qarth_Desc_Match'].iloc[i]
    
    jacc_sim.append(DistJaccard(inred,match_desc))
    cosine_sim.append(get_cosine(inred,match_desc))

In [ ]:
df_result['jacc_ind'] = jacc_sim
df_result['cosine_sim'] = cosine_sim

In [ ]:
df_result.head()

In [ ]:
list_ingred_un = list(df_result['Ingredients_org'].unique())

In [ ]:
#list_total
total_list = []


for ing in tqdm(list_ingred_un):
    
    df_temp = df_result[df_result['Ingredients_org'] == ing]
    
    
    #Adding elements to the list - Clean Ingredients
    ingred_clean = list(df_temp['Ingredients_cln'].unique())
    
    #Original Ingredient
    ingred_org = [ing]
    
    #Matched_upc_list
    upc_match = list(df_temp['UPC_match'])
    
    #Matched Qarth_desc
    qarth_match = list(df_temp['Qarth_Desc_Match'])
    
    #BM-25 score
    score_bm = list(df_temp['list_score_qarth'])
    
    #Appending it to list
    list_temp = [ingred_clean,ingred_org,upc_match,qarth_match,score_bm]
    
    #Combined_list
    total_list.append(list_temp)
    
    

In [ ]:
#df_result[df_result['Ingredients_org'] == 'HABANERO EXTRACT']['UPC_match']

In [ ]:
#df_result.head()
df_search = pd.DataFrame()
df_search['res_search'] = total_list

In [ ]:
total_list[0][1]

In [ ]:
df_search.to_excel('/Users/s0c02nj/Desktop/search_result.xlsx')

In [ ]:
## Order
# ingred_clean
# ingred_orginal
# upc_match
# qarth_match ---> Description of Match
# score_bm25


#### Statistical Analysis

In [ ]:
np.corrcoef(df_result['list_score_qarth'].tolist(),df_result['jacc_ind'].tolist())

In [ ]:
np.corrcoef(df_result['list_score_qarth'].tolist(),df_result['cosine_sim'].tolist())

In [ ]:
np.corrcoef(df_result['jacc_ind'].tolist(),df_result['cosine_sim'].tolist())

In [ ]:
df_result['cosine_sim'].mean()

In [ ]:
#sns.distplot(df_result['list_score_qarth'])
sns.distplot(df_result['list_score_qarth'].tolist())

In [ ]:
#sns.distplot(df_result['list_score_qarth'])
sns.distplot(df_result['cosine_sim'].tolist())

In [ ]:
#Rank Based on Similarity measures --bm25
df_result['rank_bm25_score'] = df_result.groupby('Ingredients_cln')['len_char'].rank(ascending=False,method ='dense')

#Rank Based on Similarity measures --cosine
df_result['rank_cos_score'] = df_result.groupby('Ingredients_cln')['len_char'].rank(ascending=False,method ='dense')

#Rank Based on Similarity measures --jaccard
df_result['rank_jac_score'] = df_result.groupby('Ingredients_cln')['len_char'].rank(ascending=False,method ='dense')

In [5]:
df_ingred = pd.read_pickle('/Users/s0c02nj/Downloads/df_ingred')

In [7]:
df_ingred['Ingredients_cln'].unique().shape

(29566,)

In [11]:
df_ingred['Ingredients'].unique().shape

(31033,)